In [3]:
import sys
from os.path import join
from pathlib import Path
parent_path = str(Path.cwd().parents[0])

if parent_path not in sys.path:
    sys.path.append(parent_path)

from src import manual_preprocessing as mp
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dataset = pd.read_csv(join(mp.data_path, 'data.csv'))


# Analisis exploratorio de los datos
En esta sección vamos a abordar la primera fase del proceso del analisis de datos. Vamos a estudiar a fondo el dataset que se nos ha entregado y comenzar a conocer el problema y hacer un procesamiento manual de los datos.




In [16]:
dataset.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672 entries, 0 to 1671
Data columns (total 200 columns):
 #    Column                   Dtype  
---   ------                   -----  
 0    Curso                    float64
 1    Nota                     float64
 2    Beca                     object 
 3    Género                   object 
 4    Edad                     float64
 5    Trabajas                 object 
 6    TrabAnt                  object 
 7    EmpFam                   object 
 8    Emprend                  object 
 9    EmpFut                   object 
 10   SE1                      float64
 11   SE2                      float64
 12   SE3                      float64
 13   SE4                      float64
 14   SE5                      float64
 15   SE6                      float64
 16   SEMedia                  float64
 17   AC1                      float64
 18   AC2                      float64
 19   AC3                      float64
 20   AC4                      flo

### Comprobacion de columnas con mismos datos

Tras una exhauctiva lectura de la explicacion de cada variable, me  he dado cuenta que hay un grupo de columnas que son problematicas, estas columnas son las BFx y las BFxAdaptada. 

En las columnas BFx tenemos unicamente datos que se recogieron en el año `19_20` y en las columnas BFxAdaptada tenemos datos que se recogieron en `19_20` y en `18_19`. Esto es un problema ya que puede ser que haya datos que no se corresponden entre las dos columnas. Para comprobarlo he creado la funcion de abajo a la que se le dan todos los datos y comprueba si dos columnas son iguales.

In [5]:
# Lectura de datos
list_adaptada = ['BF1Adaptada', 'BF2Adaptada', 'BF3Adaptada', 'BF4Adaptada']
list_no_adaptada =  ['BF1', 'BF2', 'BF3', 'BF4']
res_data = mp.compare_columns(dataset, list_adaptada, list_no_adaptada, 'Año', '19_20', int)

if res_data is not None:
    print(f"Las columnas son iguales, shape actual = {res_data.shape}")
    dataset = res_data
else:
    print("Las columnas son distintas")

Las columnas son iguales, shape actual = (1672, 246)


### Eleccion de columnas redundantes.

En esta sección, vamos a analizar una por una que columnas son redundantes y proceder a la eliminación de las mismas, ya que aunque no tenemos un dataset grande, no deberiamos tener columnas conteniendo la misma informacion.

Vamos a comenzar por borrar algunas de las columnas `CF2`, `CF4`, `CF5` ya que son respuestas libres. Intentar categorizar este tipo de columnas implicaria un gran coste, debido al número de posibles respuestas de cada uno de los encuestados. Ademas, ya tenemos en las columas QFx valores 0,1 si las preguntas anteriores estan contestadas bien. Las columnas que se van a dejar son: 

- `CF1`: Esta se deja ya que es una pregunta en escala Likert 
- `CF3`: Esta es una pregunta con opciones. La columna `QF3` asociada tambien se va a dejar, ya que indica si la respuesta es correcta o no. Esta columna tiene interés.
- `CF6`: Igual que `CF3`

Las variables `BFx` representan la respuesta a una pregunta de la encuesta. Tenemos dos de esas preguntas que son de verdadero o falso, asi que esas columnas van a ser borradas. El resto, son preguntas con opciones, y estas columnas pueden ser interesantes. Las columnas que se van a borrar son:

- `Los dividendos son parte de lo que una empresa paga al banco para devolver un préstamo.`
- `Cuando una empresa obtiene capital de un inversor, le está dando al inversor parte de la propiedad de la compañía.`

Las columnas `BA2` y `QF2` son redundantes. `BA2` contiene la respuesta de una pregunta de tipo SI/no y `QF2` tiene 1 o 0 dependendiendo de si se contesto bien o mal.

También voy a renombrar un par de columnas, ya que tienen un nombre muy largo (la pregunta entera que se hizo en la encuesta). Esto no va a influir en el algoritmo, pero me es más cómodo trabajar con columnas con nombres pequeños.


In [6]:
dataset.drop(columns=mp.columns_to_delete, inplace=True)
dataset.rename(mapper=mp.rename_dict, inplace=True)
# There are values like spaces that we don't want, as they introduce some noise into the model
dataset.replace(to_replace=' ', value=np.nan, inplace=True)


Probando los valores únicos en las columnas, me di cuenta que la columna de género tiene géneros como `sí` y `no`. Estos valores no son válidos y antes de introducirlos así al algoritmo, vamos a considerar estos valores como valores perdidos, ya que no son validos. Lo hacemos justo antes de buscar los datos perdidos, si vemos que tras esta transformacion la columna contiene una gran cantidad de valores perdidos, se eliminará.

In [7]:
print(dataset['Género'].unique())
dataset['Género'].replace(to_replace='Sí', value=np.nan, inplace=True)
dataset['Género'].replace(to_replace='No', value=np.nan, inplace=True)

['Hombre' 'Mujer' nan 'No' 'Sí']


### Eliminación de columnas con gran cantidad de valores perdidos.

Revisando las columnas, he visto que hay varias columnas que solo contienen datos de un solo año. Esto es un problema, ya que dichas estas columnas van a tener una gran cantidad de valores perdidos. Imputar valores en dichas columnas no es una opción, ya que podemos introducir una gran cantidad de ruido en el dataset. Lo mejor es eliminarlas. 
A continuación se muestra cuales son dichas columnas.

In [8]:
lost_values = mp.get_missing_values(dataset, 0.5)
print(f"Columnas con más del 50% de valores perdidos:\n'{lost_values}'")

dataset.drop(columns=lost_values, inplace=True)

Columnas con más del 50% de valores perdidos:
'Index(['Si el efectivo al final de un determinado periodo (día, mes, año,...) es mayor que al comienzo del periodo, significa que la empresa ha generado un beneficio positivo.',
       'Una empresa acaba de adquirir un bien de equipo por el que ha pagado 200 euros. El bien será utilizado durante 5 años. El beneficio del ejercicio actual se verá reducido en:',
       'El balance de situación es:',
       '¿Cuál de las siguientes opciones describe mejor el ratio de rentabilidad sobre los activos (ROA)? ',
       'No tener deuda es siempre una situación deseable para una empresa.',
       'Cuando las ventas aumentan, significa que la empresa goza de buena salud.',
       'La rentabilidad sobre activos se conoce como ROA, y la rentabilidad sobre los fondos propios invertidos por los accionistas en la empresa se conoce como ROE. Por regla general, el nivel de deuda es más sostenible si:',
       'TotalEF (1-8)', 'EFB (1-3)', 'SumaEFA1_4', 'Suma

### Correlacion de columnas
Vamos a calcular la correlacion que hay entre las columnas del dataset. Esto nos permitirá eliminar aquellas columnas que tengan una correlación alta.
A continuación, se muestran las 20 columnas con una correlación más alta.

Antes de seguir, vamos a separar las columnas que se van a usar como predictoras. 

In [9]:
# Get the predictors data
predictor_data = dataset[mp.predictors_name]
dataset.drop(columns=mp.predictors_name, inplace=True)

correlation_matrix = dataset.corr().abs()
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)).unstack().sort_values(kind="quicksort", ascending=False)
print(upper[:20])

FI_Insur                 IF1.6jTiene              1.000000
FinLit normaliz          FinLit total (1-19)      1.000000
ConFin (1_8)             ConFin (1_7)             0.987110
FinBeh con borrow (1-7)  FinBehSinBorrow (1-6)    0.948779
NSMedia                  NS2                      0.918361
FI_Aware                 SumaConoces              0.892417
NSMedia                  NS3                      0.891036
SumaConoces              IF1.6iConoce             0.886519
FI_Aware                 IF1.6cConoces            0.886250
SumaConoces              IF1.6lConoce             0.878140
                         IF1.6kConoce             0.877966
AcMedia                  AC2                      0.877745
                         AE5                      0.874652
IF1.6lConoce             IF1.6kConoce             0.873608
SEMedia                  SE3                      0.873497
NSMedia                  NS1                      0.863767
AcMedia                  AC3                      0.8636

Podemos ver que hay dos pares de columnas con una correlacion máxima: 
- `F1_Insur` y `IF1.6jTiene`: La columna `IF1.6jTiene` contiene informacion sobre si tienen un contrato de seguro y la columna `F1_Insur` va a ser 1 cuando tenga un seguro contratado. Asi que `F1_Insur` va a ser borrada.
- `FinLit normaliz` y `FinLit total`: Respecto a estas dos, `FinLit normaliz` contiene la columna `FinLit` normalizada. Se va a borrar `FinLit normaliz`
- `ConFin (1_8)` y `ConFin (1_7)` son columnas que contienen la suma de otras columnas, asi que ambas podrían ser eliminadas.
También vamos a borrar unas columnas que contienen la suma y la media de ciertas columnas, ya que no proporcionan ninguna información.


In [10]:
corr_cols = mp.get_highly_correlated_columns(dataset, 0.95)
dataset.drop(columns=corr_cols, inplace=True)


3 features are highly correlated. They will be removed: ['ConFin (1_8)', 'FinLit normaliz', 'FI_Insur']


### Comprobar balance de clases
Vamos a comprobar el balance de clases en las columnas que se van a usar como predictores. Necesitamos saber si estamos ante un problema de clasificación desbalanceado o no.
Esta comprobación solo la vamos a hacer en las columnas que se van a usar para clasificación, ya que no tiene sentido ver la distribucion de clases para la variable que se usara para regresión.

In [11]:
%matplotlib ipympl
fig = mp.print_value_occurrences(predictor_data[mp.classification_predictor])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

En la gráfica de arriba, podemos ver que si que tenemos un desbalance de clases respecto a los valores `1.0`, `2.0`, `3.0`  en todas las variables predictoras. 
Este problema lo podemos afrontar realizando un `subsampling` en aquellas variables dominantes.
